In [ ]:
!pip install groq
!pip install faster-whisper
!apt-get install portaudio19-dev
!pip install pyaudio
!pip install speechrecognition


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additi

In [ ]:
!pip install Pillow

!pip install flask pyngrok flask-cors

!apt-get install ffmpeg

!apt-get install ngrok

!pip install pydub
#!ngrok http 5000


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ngrok


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import os
import re
from groq import Groq
from PIL import ImageGrab, Image
import cv2
import pyperclip
import google.generativeai as genai
from openai import OpenAI
import pyaudio
from faster_whisper import WhisperModel
import speech_recognition as sr
import time
import json
from PIL import Image

# Initialize Flask app and CORS
app = Flask(__name__)
CORS(app)

# Set up ngrok
ngrok.set_auth_token('')  # Replace with your actual auth token
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))

# Initialize models and settings
groq_client = Groq(api_key="")  # Replace with your Groq API key
genai.configure(api_key='')  # Replace with your Generative AI API key
openai_client = OpenAI(api_key='')  # Replace with your OpenAI API key
whisper_model = WhisperModel("base", device='cpu', compute_type='int8')

# System message for AI
sys_msg=(
    'You are a multi-modal AI voice assistant. Your user may or may not have attached a photo for context '
    '(either a screenshot or a webcam capture). Any photo has already been processed into a hoghly detailed '
    'text prompt that will be attached to their transcribed voice prompt. Generate the most useful and '
    'factual response possible, carefully considering all previous generated text in your response before '
    'adding new tokens to the response. Do not expect or request images, just use the context if added. '
    'Use all of the context of this conversation so your response is relevant to the conversation. Make '
    'your responses clear and concise, avoiding any verbosity.'
)

convo = [{'role': 'system', 'content': sys_msg}]


generation_config = {
    'temperature': 0.7,
    'top_p': 1,
    'top_k': 1,
    'max_output_tokens': 2048
}

safety_settings = [
    {
        'category': 'HARM_CATEGORY_HARASSMENT',
        'threshold': 'BLOCK_NONE'
    },
    {
        'category': 'HARM_CATEGORY_HATE_SPEECH',
        'threshold': 'BLOCK_NONE'
    },
    {
        'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
        'threshold': 'BLOCK_NONE'
    },
    {
        'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
        'threshold': 'BLOCK_NONE'
    },
]

model = genai.GenerativeModel('gemini-1.5-flash-latest',
                              generation_config=generation_config,
                              safety_settings=safety_settings)

num_cores = os.cpu_count()
whisper_size = 'base'
whisper_model = WhisperModel(
    whisper_size,
    device='cpu',
    compute_type='int8',
    cpu_threads=num_cores // 2,
    num_workers=num_cores // 2
)

def groq_prompt(prompt, img_context):
    if img_context:
        prompt = f'USER PROMPT: {prompt}\n\n  IMAGE CONTEXT: {img_context}'
    convo.append({'role': 'user', 'content': prompt})
    chat_completion = groq_client.chat.completions.create(messages=convo, model='llama3-70b-8192')
    response = chat_completion.choices[0].message
    convo.append(response)
    return response.content

'''
def take_screenshot():
    path = 'screenshot.jpg'
    screenshot = ImageGrab.grab()
    rgb_screenshot = screenshot.convert('RGB')
    rgb_screenshot.save(path, quality=15)

def web_cam_capture():
    if not web_cam.isOpened():
        print('Error: Camera did not open successfully')
        exit()
    path = 'webcam.jpg'
    ret, frame = web_cam.read()
    cv2.imwrite(path, frame)
'''
def get_clipboard_text():
    clipboard_content = pyperclip.paste()
    if isinstance(clipboard_content, str):
        return clipboard_content
    else:
        print('No Clipboard text to copy')
        return None

def vision_prompt(prompt, photo_path):
    img = Image.open(photo_path)
    prompt = (
        'You are the vision analysis AI that provides semantic meaning from images to provide context '
        'to send to another AI that will create a response to the user. Do not respond as the AI assistant '
        'to the user. Instead take the user prompt input and try to extract all meaning from the photo '
        'relevant to the user prompt. Then generate as much objective data about the image for the AI '
        f'assistant who will respond to the user. \nUSER PROMPT: {prompt}'
    )
    response = model.generate_content([prompt, img])
    return response.text

def function_call(prompt):
    sys_msg = (
        'You are an AI function calling model. You will determine whether extracting the user\'s clipboard content, '
        'taking a screenshot, capturing the webcam, or calling no functions is best for a voice assistant to respond '
        'to the user\'s prompt. The webcam can be assumed to be a normal laptop webcam facing the user. You will '
        'respond with only one selection from this list: ["extract clipboard", "take screenshot", "capture webcam", "None"] \n'
        'function call name exactly as I listed.'
    )

    function_convo = [{'role': 'system', 'content': sys_msg},
                      {'role': 'user', 'content': prompt}]
    chat_completion = groq_client.chat.completions.create(messages=function_convo, model='llama3-70b-8192')
    response = chat_completion.choices[0].message
    return response.content

def wav_to_text(audio_path):
    segments, _ = whisper_model.transcribe(audio_path)
    text = ''.join(segment.text for segment in segments)
    return text

def extract_prompt(transcribed_text, wake_word):
    pattern = rf'\b{re.escape(wake_word)}[\s,.?!]*([A-Za-z0-9].*)'
    match = re.search(pattern, transcribed_text, re.IGNORECASE)

    if match:
        prompt = match.group(1).strip()
        return prompt
    else:
        return None

# Function to process audio and generate responses
def process_audio_file(audio_file_path):
    # Read and transcribe the audio file
    print('Processing audio file:', audio_file_path)

    prompt_text = wav_to_text(audio_file_path)
    print('Transcribed text:', prompt_text)

    clean_prompt = extract_prompt(prompt_text, "hey")
    print('Clean prompt:', clean_prompt)

    if clean_prompt:
        call = function_call(clean_prompt)
        print('Function call response:', call)

        visual_context = None  # Initialize visual context variable
        '''
        if 'take screenshot' in call:
            take_screenshot()
            visual_context = vision_prompt(prompt=clean_prompt, photo_path='screenshot.jpg')
        elif 'extract clipboard' in call:
            paste = get_clipboard_text()
            prompt = f'{clean_prompt}\n\n CLIPBOARD CONTENT: {paste}'
        '''
        response = groq_prompt(prompt=clean_prompt, img_context=visual_context)
        return response
    else:
        print('No valid clean prompt found.')
        return None

@app.route("/upload", methods=["POST"])
def upload_audio():
    if "file" not in request.files:
        return jsonify({"error": "No file part"}), 400

    file = request.files["file"]
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Save the uploaded file temporarily
    file.save("uploaded_audio.wav")

    # Process the audio file and return the response
    response = process_audio_file("uploaded_audio.wav")

    if response:
        return jsonify({"response": response})
    else:
        return jsonify({"error": "Could not process audio"}), 500

# Add new endpoint to process the image file
@app.route("/upload_image", methods=["POST"])
def upload_image():
    if "file" not in request.files:
        return jsonify({"error": "No file part"}), 400

    file = request.files["file"]
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Save the uploaded image temporarily
    image_path = "uploaded_image.jpg"
    file.save(image_path)

    # Process the image file (for example, using your vision_prompt function)
    visual_context = vision_prompt(prompt="Analyze this image", photo_path=image_path)

    if visual_context:
        return jsonify({"response": visual_context})
    else:
        return jsonify({"error": "Could not process image"}), 500

@app.route("/upload_screenshot", methods=["POST"])
def upload_screenshot():
    if "file" not in request.files:
        print("No file part in the screenshot request")
        return jsonify({"error": "No file part"}), 400

    file = request.files["file"]
    if file.filename == '':
        print("No selected screenshot file")
        return jsonify({"error": "No selected file"}), 400

    # Save the uploaded screenshot temporarily
    screenshot_path = "uploaded_screenshot.jpg"
    file.save(screenshot_path)
    print("Screenshot saved successfully:", screenshot_path)

    # Process the screenshot using vision_prompt
    visual_context = vision_prompt(prompt="Analyze this screenshot", photo_path=screenshot_path)
    print("Screenshot processing response:", visual_context)

    if visual_context:
        return jsonify({"response": visual_context})
    else:
        return jsonify({"error": "Could not process screenshot"}), 500

# Start Flask app
if __name__ == "__main__":
    app.run()
